In [6]:
import numpy as np
from observables import observables
from k_space import k_space


# Minimal p-Hamiltonian
$p_z,p_x,p_y$-Hamiltonian, with Dirac point in the $p_x,p_y$ bands, gapped by $L_z$
\begin{aligned}
    H = E_{p_z} + k_x \cdot s_x + k_y \cdot s_z + \lambda \cdot s_y
\end{aligned}

In [34]:
s = np.zeros((4,2,2),dtype=complex)
s[0] = np.eye(2)
s[1] = np.array([[0,1],[1,0]])
s[2] = 1j*np.array([[0,-1],[1,0]])
s[3] = np.array([[1,0],[0,-1]])

class dirac_ham:
    def __init__(self,L):
                
        self.l      = L
        self.E_pz   = -10
        self.bra_vec= np.array([[1,0,0],[0,1,0],[0,0,1]])
        self.basis  = np.array([1])
        self.ef     = 0
        self.n_elec = 2
        self.spin   = False
        self.n_orb  = 3
        self.n_bands= 3

    def hk(self,k_red):
        d = np.zeros((k_red.shape[0],4))
        d[:,1] = k_red[:,0]
        d[:,3] = k_red[:,1]
        d[:,2] = self.l
        hk_out = np.zeros((k_red.shape[0],3,3),dtype=complex)
        hk_out[:,0,0] = self.E_pz
        hk_out[:,1:,1:] = np.einsum('ij,jkl->ikl',d,s)
        return hk_out
        
    def del_hk(self,k_red):
        d = np.zeros((k_red.shape[0],3,4))
        d[:,0,1] = 1
        d[:,1,3] = 1
        del_hk_out = np.zeros((k_red.shape[0],3,3,3),dtype=complex)
        del_hk_out[:,:,1:,1:] = np.einsum('idj,jkl->idkl',d,s)
        print(del_hk_out.shape)
        return del_hk_out





In [35]:
ktype  = "path"
kbasis = "red"
vecs   = np.array([[-1,0,0],[1,0,0]])
npoints = 100
bra_vec = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_space = k_space(ktype,kbasis,vecs,bra_vec,npoints)

In [36]:
l=1
ham = dirac_ham(l)
op_types = ["L"]
op_types_k = ["BC"]
Observables = observables(ham,K_space,op_types,op_types_k)
Observables.calculate_ops()

Initializing k-independent operator L.
Inititalizing k-dependent operator BC.
Calculating operators on the given k-space...
Diagonalizing all k-points in parallel.
Time for running H(k) FT: 0.0002970695495605469
Time for diagonalizing H(k): 0.00039196014404296875
Time for calculating expectation value of operator L: 0.0015940666198730469
(100, 3, 3, 3)
Time for calculating expectation value of operator BC: 0.002888917922973633
Shifting eigenvalues w.r.t. Fermi level...
Running post-processing for operator L.
Running post-processing for operator BC.
Writing eigenvalues output.
Writing output for operator L.
Writing output for operator BC.
Writing band-integrated output for operator BC.


# Calculation of the $z$-component of the valence BC by hand at k=0

In [73]:
k = np.array([[0,0,0]])
hk = ham.hk(k)
evals,evecs = np.linalg.eigh(hk)

#valene and conduction eigenvectors
v = evecs[:,1:,1]
c = evecs[:,1:,2]
# z-componend of the valence state
vsxc = np.einsum("i,i",v.conjugate()[0],np.einsum("ij,j",s[1],c[0]))
cszv = np.einsum("i,i",c.conjugate()[0],np.einsum("ij,j",s[3],v[0]))

omega_z = 2*1j *vsxc*cszv/(evals[0,1]-evals[0,2])**2
print("Valence band Berry curvature:",omega_z)

Valence band Berry curvature: (-0.4999999999999998+0j)
